In [ ]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets

import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')

import ballsbot.config as config
config.ENABLE_MULTIPROCESSING = False

In [ ]:
import ballsbot.drawing as drawing
from ballsbot.lidar import calibration_to_xywh
from ballsbot.drawing import BotPoseAbsCoordsDrawing
from ballsbot.utils import keep_rps
from ballsbot.lidar import revert_transformation_to_cloud, apply_transformation_to_cloud
from ballsbot.dashboard import Dashboard

In [ ]:
only_nearby_meters = 4

In [ ]:
new_format = True
with open('track_2024-08-20.json') as hf:
    if new_format:
        track = []
        for line in hf:
            try:
                frame = json.loads(line)
                track.append(frame)
            except Exception as e:
                print(f'broken frame {e}')
    else:
        track = json.loads(hf.read())

In [ ]:
calibration = {
    'angle_fix': 3.0810609316737434,  # lidar orientation (radians)
    'fl_x': 0.2041686441507201,  # front left corner coords
    'fl_y': 0.10426277741236079,
    'rr_x': -0.08011094659163859,  # rear right corner coords
    'rr_y': -0.0988675829003773,
}
self_position = calibration_to_xywh(calibration)


dashboard = Dashboard(widgets, image_size=1000)

augmented_painter = BotPoseAbsCoordsDrawing(dashboard, 'augmented')
dashboard.add_subplot('augmented', augmented_painter.get_drawing_func_name())

display(dashboard.get_image())

In [ ]:
# poses = []
# additional_points = []
# ts = None
# for frame_index, track_frame in enumerate(track):
#     if not track_frame['tracker']['current_pose']:
#         continue
#     ts = keep_rps(ts, fps=1)
#     poses.append(track_frame['tracker']['current_pose'])

#     if track_frame['lidar']['augmented_points']:
#         points = apply_transformation_to_cloud(
#             track_frame['lidar']['augmented_points'],
#             [track_frame['tracker']['current_pose'][v] for v in ('x', 'y', 'teta')]
#         )
#         additional_points += [x for i, x in enumerate(points) if i % 20 == 0]
#         augmented_painter.update_image(
#             poses,
#             points,
#             self_position,
#             only_nearby_meters=only_nearby_meters,
#             additional_points=additional_points,
#         )
        
#         dashboard.redraw()

#     clear_output(wait=True)
#     print(frame_index)

In [ ]:
from python_pcl_ndt import python_pcl_ndt

In [ ]:
poses = []
additional_points = []
ts = None
for frame_index, track_frame in enumerate(track):
    if not track_frame['tracker']['current_pose']:
        continue
    ts = keep_rps(ts, fps=1)

    if track_frame['lidar']['augmented_points']:
        result = python_pcl_ndt.align_clouds(
            track_frame['tracker']['current_pose'],
            track_frame['lidar']['augmented_points'],
            False
        )
        poses.append({
            'x': result['x'],
            'y': result['y'],
            'teta': result['teta'],
        })

        points = apply_transformation_to_cloud(
            track_frame['lidar']['augmented_points'],
            [poses[-1][v] for v in ('x', 'y', 'teta')]
        )
        additional_points += [x for i, x in enumerate(points) if i % 20 == 0]
        augmented_painter.update_image(
            poses,
            points,
            self_position,
            only_nearby_meters=only_nearby_meters,
            additional_points=additional_points,
        )
        
        dashboard.redraw()
    else:
        poses.append(track_frame['tracker']['current_pose'])

    clear_output(wait=True)
    print(frame_index)